In [1]:
import this

The Zen of Python, by Tim Peters

Beautiful is better than ugly.
Explicit is better than implicit.
Simple is better than complex.
Complex is better than complicated.
Flat is better than nested.
Sparse is better than dense.
Readability counts.
Special cases aren't special enough to break the rules.
Although practicality beats purity.
Errors should never pass silently.
Unless explicitly silenced.
In the face of ambiguity, refuse the temptation to guess.
There should be one-- and preferably only one --obvious way to do it.
Although that way may not be obvious at first unless you're Dutch.
Now is better than never.
Although never is often better than *right* now.
If the implementation is hard to explain, it's a bad idea.
If the implementation is easy to explain, it may be a good idea.
Namespaces are one honking great idea -- let's do more of those!


In [2]:
from math import pi

from ipycanvas import Canvas

canvas = Canvas(width=1600, height=1200, layout=dict(width="10%"))

canvas.fill_style = "#8ee05e"
canvas.fill_rect(0, 0, canvas.width, canvas.height)

canvas.fill_style = "#f5f533"
canvas.fill_circle(canvas.width / 2.0, canvas.height / 2.0, 500)

canvas.stroke_style = "black"
canvas.line_width = 30
canvas.stroke_circle(canvas.width / 2.0, canvas.height / 2.0, 500)

canvas.fill_style = "black"
canvas.fill_circle(canvas.width / 2.7, canvas.height / 3.0, 100)  # Right eye
canvas.stroke_arc(canvas.width / 2.0, canvas.height / 2.0, 400, 0, pi, False)  # Mouth
canvas.stroke_arc(
    canvas.width - canvas.width / 2.7, canvas.height / 2.7, 100, 0, pi, True
)  # Left eye

canvas

Canvas(height=1200, layout=Layout(width='10%'), width=1600)

In [3]:
"Plain NumPy implementation of simulation in fused.ispc.c."

import numpy as np
import scipy.sparse
import tqdm


def _cfun(t, buffer, csr_weights, idelays2, horizon):
    cx = buffer[csr_weights.indices, (t-idelays2) % horizon]
    cx *= csr_weights.data.reshape(-1, 1)
    cx = np.add.reduceat(cx, csr_weights.indptr[:-1], axis=1)
    return cx  # (2, num_node, num_item)

def _dfun(x, cx, sim_params):
    a, tau, k = sim_params
    return np.array([
        tau*(x[0] - x[0]**3/3 + x[1]),
        (1/tau)*(a + k*cx - x[0])
    ])  # (2, num_node, num_item)

def _heun(x, cx, dt, num_node, num_item, dfun, sim_params):
    z = np.random.randn(2, num_node, num_item)
    z *= 0  # z_scale.reshape((2, 1, 1))
    dx1 = dfun(x, cx[0], sim_params)
    dx2 = dfun(x + dt*dx1 + z, cx[1], sim_params)
    return x + dt/2*(dx1 + dx2) + z


def run_sim_np(
    csr_weights: scipy.sparse.csr_matrix,
    idelays: np.ndarray,
    sim_params: np.ndarray,
    z_scale: np.ndarray,
    horizon: int,
    rng_seed=43, num_item=8, num_node=90, num_svar=2, num_time=1000, dt=0.1,
    num_skip=5
):
    trace_shape = num_time // num_skip + 1, num_svar, num_node, num_item
    trace = np.zeros(trace_shape, 'f')
    assert idelays.max() < horizon-2
    idelays2 = -horizon + np.c_[idelays, idelays-1].T
    assert idelays2.shape == (2, csr_weights.nnz)
    buffer = np.zeros((num_node, horizon, num_item))

    x = np.zeros((2, num_node, num_item), 'f')

    for t in tqdm.trange(trace.shape[0]):
        for tt in range(num_skip):
            ttt = t*num_skip + tt
            cx = _cfun(ttt, buffer, csr_weights, idelays2, horizon)
            x = _heun(x, cx, dt, num_node, num_item, _dfun, sim_params)
            buffer[:, ttt % horizon] = x[0]
        trace[t] = x

    return trace


In [6]:
import time

np.random.seed(42)

num_item = 64
num_node = 90
num_skip = 10
dt = 0.1
sparsity = 0.3
horizon = 256
num_time = int(1e3/dt)
horizonm1 = horizon - 1
sim_params = np.zeros((3, num_item), 'f')
sim_params[0] = 1.001
sim_params[1] = 1.0
sim_params[2] = np.logspace(-1.8, -2.0, num_item)/num_node*80 # k
z_scale = np.sqrt(dt)*np.r_[0.01, 0.1].astype('f')*1e-8

weights, lengths = np.random.rand(2, num_node, num_node).astype('f')
lengths[:] *= 0.8
lengths *= (horizon*dt*0.8)
zero_mask = weights < (1-sparsity)
weights[zero_mask] = 0
csr_weights = scipy.sparse.csr_matrix(weights)
idelays = (lengths[~zero_mask]/dt).astype('i')+2

tic = time.time()
run_sim_np(
    csr_weights, idelays, sim_params, z_scale, horizon,
    num_item=num_item, num_node=num_node, num_time=num_time,
    dt=dt, num_skip=num_skip
)
tok = time.time()
print(tok - tic, 's', num_time*num_item/(tok-tic), 'iter/s numpy')

100%|██████████| 1001/1001 [00:07<00:00, 136.56it/s]

7.332791566848755 s 87279.1752179911 iter/s numpy


In [9]:
np.save('weights.npy',
        np.random.randn(num_node, num_node).astype('f') * 0.01)

In [10]:
weights = np.load('weights.npy')
print(weights.shape)

(90, 90)


In [11]:
import ipywidgets as widgets

In [12]:
slider = widgets.IntSlider(value=0, min=0, max=100)
slider

IntSlider(value=0)

In [13]:
import mdn
mdn.test_mdn()

Training MDN on 4000 samples...
Input features dimension: 1, Output parameters dimension: 2

Starting training...
Iteration     1/1200, Loss: 8.8426
Iteration   200/1200, Loss: 2.0169
Iteration   400/1200, Loss: 1.9614
Iteration   600/1200, Loss: 1.9401
Iteration   800/1200, Loss: 1.9266
Iteration  1000/1200, Loss: 1.9179
Iteration  1200/1200, Loss: 1.9139
Training finished.

Final loss: 1.9139

--- Validation ---
True Mode 1 (for positive feats): [2. 3.]
True Mode 2 (for negative feats): [-2. -3.]
Predicted mode for positive feature: [2.04 3.04]
Predicted mode for negative feature: [-2.01 -3.01]

Success: The network correctly identified the true modes of the distribution.
